In [76]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from thingspeak_reader import * 
from sklearn.model_selection import train_test_split

import cherrypy
import json


In [44]:
with open("D:\\GitHub\\SmartGreenHouse\\SmartGreenhouse\\Data_analysis\\conf_DA.json") as f:
    confDA = json.load(f)

csv_data = confDA.get("path_dataset1")
df1 = pd.read_csv(csv_data, sep=";", decimal=',')

In [45]:
df1.columns

Index(['id', 'greenhous_temperature_celsius', 'greenhouse_humidity_percentage',
       'greenhouse_illuminance_lux', 'online_temperature_celsius',
       'online_humidity_percentage',
       'greenhouse_total_volatile_organic_compounds_ppb',
       'greenhouse_equivalent_co2_ppm', 'created'],
      dtype='object')

In [46]:
df1 = df1.drop(columns=['greenhouse_illuminance_lux', 'online_temperature_celsius',
       'online_humidity_percentage', 'greenhouse_total_volatile_organic_compounds_ppb'])

In [47]:
df1 = df1.dropna()

In [48]:
df1.isna().any().any()

False

In [49]:
df1.isna().sum()

id                                0
greenhous_temperature_celsius     0
greenhouse_humidity_percentage    0
greenhouse_equivalent_co2_ppm     0
created                           0
dtype: int64

In [51]:
df1.describe()

,id,greenhous_temperature_celsius,greenhouse_humidity_percentage,greenhouse_equivalent_co2_ppm
count,1.716400e+04,17164.000000,17164.000000,17164.000000
mean,7.895066e+05,25.874693,52.910666,1219.092382
std,2.441935e+05,13.038214,13.079384,1234.655687
min,3.614960e+05,4.972237,15.313954,400.000000
25%,5.783872e+05,14.951478,45.488623,424.000000
50%,7.930140e+05,24.121462,55.613870,985.950000
75%,9.978682e+05,31.744516,63.771343,1385.925000
max,1.206788e+06,77.566312,82.900305,18113.400000


In [53]:
# deal with the second dataset, the labeled one
with open("D:\\GitHub\\SmartGreenHouse\\SmartGreenhouse\\Data_analysis\\conf_DA.json") as f:
    confDA = json.load(f)

csv_data = confDA.get("path_dataset2")
df2 = pd.read_csv(csv_data, sep=";", decimal='.')

In [56]:
df2

,HS (Analog),L (Lux),T (°),CO2 (Analog),HR (%),clase
0,713,0.83,16.2,76,96.8,1
1,717,2.50,16.2,75,96.5,1
2,717,561.67,16.1,78,96.7,1
3,716,640.00,15.9,79,96.5,1
4,718,568.33,16.1,79,96.6,1
...,...,...,...,...,...,...
295,550,280.83,21.0,153,86.7,4
296,549,260.83,20.8,151,84.3,4
297,550,215.00,20.6,148,86.3,4
298,549,215.83,20.4,151,87.8,4


In [55]:
df2.isna().any().any() # there are not NaN values

False

In [60]:
df2 = df2.drop(columns=["L (Lux)"])


KeyError: "['L (Lux)'] not found in axis"

In [62]:
df2.columns

Index(['HS (Analog)', 'T (°)', 'CO2 (Analog)', 'HR (%)', 'clase'], dtype='object')

In [68]:
df1 = df1.rename(columns={'greenhous_temperature_celsius':'T (°)', 'greenhouse_humidity_percentage':'HR (%)', 'greenhouse_equivalent_co2_ppm':'CO2 (Analog)', 'created': 'timestamp'})

In [73]:
df1.describe()

,id,T (°),HR (%),CO2 (Analog)
count,1.716400e+04,17164.000000,17164.000000,17164.000000
mean,7.895066e+05,25.874693,52.910666,1219.092382
std,2.441935e+05,13.038214,13.079384,1234.655687
min,3.614960e+05,4.972237,15.313954,400.000000
25%,5.783872e+05,14.951478,45.488623,424.000000
50%,7.930140e+05,24.121462,55.613870,985.950000
75%,9.978682e+05,31.744516,63.771343,1385.925000
max,1.206788e+06,77.566312,82.900305,18113.400000


In [74]:
df2.describe()

,HS (Analog),T (°),CO2 (Analog),HR (%),class
count,300.000000,300.000000,300.000000,300.000000,300.000000
mean,603.676667,24.008867,137.506667,79.002000,2.263333
std,168.878869,4.907647,41.876830,14.974156,1.188288
min,124.000000,15.900000,34.000000,39.200000,1.000000
25%,474.500000,20.575000,91.000000,72.075000,1.000000
50%,569.500000,22.450000,149.000000,83.650000,2.000000
75%,775.000000,28.400000,162.000000,90.025000,3.000000
max,821.000000,40.200000,243.000000,96.900000,4.000000


In [78]:
df2 = df2.rename(columns={'clase':'class'})

df1: 17164 rows × 5 columns

    id: measurement id 
    T(°) : greenhouse temperature
    HR (%) : Relative greenhouse humidity
    CO2(Analog): greenhouse CO2
    timestamp: yyyy-dd-mm hh:mm

-
df2: 300 rows × 5 columns

    HS (Analog): soil humidity
    T(°) : greenhouse temperature
    HR (%) : Relative greenhouse humidity
    CO2(Analog): greenhouse CO2
    class: 
        1: soil without water
        2: environment correct
        3: too much hot
        4: very cold

In [79]:
df2.describe()

,HS (Analog),T (°),CO2 (Analog),HR (%),class
count,300.000000,300.000000,300.000000,300.000000,300.000000
mean,603.676667,24.008867,137.506667,79.002000,2.263333
std,168.878869,4.907647,41.876830,14.974156,1.188288
min,124.000000,15.900000,34.000000,39.200000,1.000000
25%,474.500000,20.575000,91.000000,72.075000,1.000000
50%,569.500000,22.450000,149.000000,83.650000,2.000000
75%,775.000000,28.400000,162.000000,90.025000,3.000000
max,821.000000,40.200000,243.000000,96.900000,4.000000


In [81]:
target = df2['class']
target

0      1
1      1
2      1
3      1
4      1
      ..
295    4
296    4
297    4
298    4
299    4
Name: class, Length: 300, dtype: int64

In [105]:
import plotly.express as px

# Assuming 'df' is your DataFrame with columns 'HS (Analog)', 'T (°)', 'CO2 (Analog)', 'HR (%)', 'class'

figHS = px.scatter(df2, x='HS (Analog)', y='class', color='class', title='Data Clustering by Class')
figHS.show()

figT = px.scatter(df2, x='T (°)', y='class', color='class', title='Data Clustering by Class')
figT.show()

figCO2 = px.scatter(df2, x='CO2 (Analog)', y='class', color='class', title='Data Clustering by Class')
figCO2.show()

figHR = px.scatter(df2, x='HR (%)', y='class', color='class', title='Data Clustering by Class')
figHR.show()

fig2 = px.scatter(df2, x='HS (Analog)', y='T (°)', color='class', title='Data Clustering by Class')
fig2.show()

fig3 = px.scatter(df2, x='HS (Analog)', y='HR (%)', color='class', title='Data Clustering by Class')
fig3.show()

fig4 = px.scatter(df2, x='HS (Analog)', y='CO2 (Analog)', color='class', title='Data Clustering by Class')
fig4.show()


In [86]:
features = df2.columns
features = features.drop('class')
features

Index(['HS (Analog)', 'T (°)', 'CO2 (Analog)', 'HR (%)'], dtype='object')

In [114]:
# Splitting the data into a training set and combined validation/test set
train_df_, test_df = train_test_split(df2, test_size=0.2, random_state=1, shuffle=True)

# Splitting the combined validation/test set into validation and test sets
train_df, val_df  = train_test_split(train_df_, test_size=0.20, random_state=1, shuffle=True)

# Printing the sizes of the resulting sets
print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

Training set size: 192
Validation set size: 48
Test set size: 60


In [133]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold


classifier = RandomForestClassifier()
pipe = Pipeline([
    ('standardization', StandardScaler()),
    ('featureS', VarianceThreshold()),
    ('ridge', classifier )
    ])
pipe.fit(train_df[features], train_df['class'])
val_predictions = pipe.predict(val_df[features])


train_score = pipe.score(train_df[features], train_df['class'])
print("Training set score:", train_score)

val_score = pipe.score(val_df[features], val_df['class'])
print("Validation set score:", val_score)

test_score = pipe.score(test_df[features], test_df['class'])
print("Test set score:", test_score)


Training set score: 1.0
Validation set score: 1.0
Test set score: 0.9833333333333333


In [128]:
from sklearn.model_selection import cross_val_score
combined_data = pd.concat([train_df, val_df])

# Perform cross-validation
cross_val_scores = cross_val_score(pipe, combined_data[features], combined_data['class'], cv=5) 

# Print the cross-validation scores
print("Cross-Validation Scores:", cross_val_scores)

# Calculate the mean cross-validation score
mean_cv_score = cross_val_scores.mean()
print("Mean Cross-Validation Score:", mean_cv_score)

Cross-Validation Scores: [0.97916667 0.9375     1.         0.97916667 1.        ]
Mean Cross-Validation Score: 0.9791666666666666


Test set score: 0.9166666666666666


In [129]:
pipe.fit(train_df_[features], train_df_['class'])
train__score = pipe.score(train_df_[features], train_df_['class'])
print("Training set score:", train__score)

test_predictions = pipe.predict(test_df[features])
print(test_predictions)

test__score = pipe.score(test_df[features], test_df['class'])
print("Test set score:", test_score)

Training set score: 1.0
[3 1 3 3 1 2 3 1 4 3 4 2 4 1 1 1 1 3 3 1 1 1 4 1 1 1 4 1 4 3 2 4 3 1 1 1 1
 3 1 3 3 3 3 1 1 3 4 3 1 3 1 4 4 3 4 1 1 3 3 3]
Test set score: 0.9833333333333333


In [ ]:
# Energy consumption analysis for the actuator
def EnergyConsumptionACT(df, ghid, devid, actid):
    consumption_time = 0
    start, stop = 0
    df_filtered = df[(df['ghID'] == ghid) & (df['devID'] == devid) & (df['actID'] == actid)]
    df_filtered.orderby(by='timestamp')

    for index, row in df_filtered.iterrows():
        if (row['v'] == 1):
            start = df_filtered['timestamp']
        elif ((row['v'] == 0) & (start != 0)):
            stop = df_filtered['timestamp']
            consumption_time += stop - start
        else:
            print('Error! Actuation is not working!')
        
    return consumption_time


In [ ]:
# Energy consumption for the device
def EnergyConsumptionDEV(df, ghid, devid):
    consumption_time_dev = 0
    df_filtered = df[(df['ghID'] == ghid) & (df['devID'] == devid)]

    for index, row in df_filtered.iterrows():
        consumption_time_dev += EnergyConsumptionACT(df, ghid, devid, row['actID'])

    return consumption_time_dev

In [ ]:
# Energy consumption for the greenhouse
def EnergyConsumptionGH(df, ghid):
    consumption_time_gh = 0
    df_filtered = df[(df['ghID'] == ghid)]
    
    for index, row in df_filtered.iterrows():
        consumption_time_gh += EnergyConsumptionDEV(df, ghid, row['devID'])

    return consumption_time_gh

In [8]:
def analysis(self, df):
    # Checking the shape of the DataFrame
    print(f"The shape of the dataframe is: {df.shape}")
    # Checking the number of missing values in each column
    print(f"The number of missing values is: {df.isnull().sum()}")
    # Handling missing data 
    # df = df.dropna(inplace=True) # drop missing values
    df = df.fillna(df.mean(), inplace=True) # fill the missing values with the mean

    # datetime handling
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    '''
    graph with data from a certain year or month or day
    '''
    df['year'] = df['timestamp'].dt.year # create year column
    df['month'] = df['timestamp'].dt.month # create month column
    df['day'] = df['timestamp'].dt.day # create day column
    # aggiungere orario

In [9]:
df1

,Data_analysis\sub\datasets\20210703_greenhouse_data.csv
